In [ ]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from diffusers import LDMPipeline, UNet2DModel, VQModel, DDIMScheduler
import torch.nn.utils.prune as prune
from tqdm import tqdm
import torch.nn.functional as F
from IPython.display import display

# ============================
# 1. Configuration Parameters
# ============================

# Paths
DATASET_PATH = "dataset/celeba_hq_256"
PRUNED_UNET_PATH = "pruned_unet_celebahq.pth"
PRUNED_VQVAE_PATH = "pruned_vqvae_celebahq.pth"
FINE_TUNED_UNET_PATH_TEMPLATE = "run/finetuned70/fine_tuned_pruned_unet_epoch_{}.pth"
FINE_TUNED_VQVAE_PATH_TEMPLATE = "run/finetuned70/fine_tuned_pruned_vqvae_epoch_{}.pth"
OUTPUT_DIR = "generated_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Pruning
PRUNE_AMOUNT_UNET = 0.5  # 30% pruning
PRUNE_AMOUNT_VQVAE = 0.5  # 30% pruning

# Training Hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
NUM_EPOCHS = 2
NUM_WORKERS = 4
IMAGE_SIZE = 256  # Assuming CelebA-HQ images are 256x256

# Device
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# ============================
# 2. Define the Custom Dataset
# ============================

class CelebaHQDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Path to the dataset directory.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.image_paths = [
            os.path.join(root_dir, img) for img in os.listdir(root_dir)
            if img.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a black image in case of error
            image = Image.new("RGB", (IMAGE_SIZE, IMAGE_SIZE), (0, 0, 0))
        if self.transform:
            image = self.transform(image)
        return image

# ============================
# 3. Prepare DataLoader
# ============================

# Define transformations (resize and normalize as needed)
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])

# Initialize the dataset and dataloader
dataset = CelebaHQDataset(root_dir=DATASET_PATH, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

# ============================
# 4. Load and Prune the Models
# ============================

# Load the pretrained models and scheduler
print("Loading pre-trained models and scheduler...")
unet = UNet2DModel.from_pretrained("CompVis/ldm-celebahq-256", subfolder="unet")
vqvae = VQModel.from_pretrained("CompVis/ldm-celebahq-256", subfolder="vqvae")
scheduler = DDIMScheduler.from_config("CompVis/ldm-celebahq-256", subfolder="scheduler")

# Pruning function for the U-Net and VQModel
def prune_model(model, amount=0.3):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            # print(f"Pruned layer: {name} - amount: {amount}")
    return model

# Apply pruning
unet = prune_model(unet, amount=PRUNE_AMOUNT_UNET)
vqvae = prune_model(vqvae, amount=PRUNE_AMOUNT_VQVAE)

# Permanently remove pruning reparameterization to make pruning effective
def remove_pruning(model):
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            prune.remove(module, 'weight')
            # print(f"Removed pruning reparameterization for {module}")

remove_pruning(unet)
remove_pruning(vqvae)


# unet.load_state_dict(torch.load(unet_path, map_location=DEVICE))
# vqvae.load_state_dict(torch.load(vqvae_path, map_location=DEVICE))
# # Save the pruned models
# torch.save(unet.state_dict(), PRUNED_UNET_PATH)
# torch.save(vqvae.state_dict(), PRUNED_VQVAE_PATH)
# print(f"Pruned models saved as {PRUNED_UNET_PATH} and {PRUNED_VQVAE_PATH}")

# Calculate and print sparsity
def calculate_sparsity(model, model_name="Model"):
    total_params = 0
    zero_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_params += param.numel()
            zero_params += (param == 0).sum().item()
    sparsity = 100.0 * zero_params / total_params
    print(f"Sparsity of {model_name}: {sparsity:.2f}% ({zero_params}/{total_params})")

calculate_sparsity(unet, "U-Net")
calculate_sparsity(vqvae, "VQ-VAE")

# ============================
# 5. Initialize the Pipeline
# ============================

# Initialize the pipeline with the pruned U-Net and VQ-VAE
print("Initializing the LDMPipeline with pruned models...")
pipeline = LDMPipeline(
    unet=unet,
    vqvae=vqvae,
    scheduler=scheduler,
).to(DEVICE)

# ============================
# 6. Define the Training Loop
# ============================

def kl_divergence_loss(noise_pred, noise):
    """
    Computes the KL Divergence between the predicted noise distribution and the actual noise.

    Parameters:
    - noise_pred (Tensor): Predicted noise by the model.
    - noise (Tensor): True Gaussian noise.

    Returns:
    - Tensor: KL Divergence loss value.
    """
    # Apply softmax to get probabilities (for demonstration, adjust if necessary)
    noise_pred_log_probs = F.log_softmax(noise_pred, dim=-1)
    noise_probs = F.softmax(noise, dim=-1)
    
    # Compute KL Divergence
    kl_loss = F.kl_div(noise_pred_log_probs, noise_probs, reduction='batchmean')
    return kl_loss

def generate_sample_images(pipeline, num_images=1, prompt="A high quality portrait"):
    pipeline.to(DEVICE)
    with torch.no_grad():
        generated_images = pipeline(batch_size=num_images, num_inference_steps=100).images
    for img in generated_images:
        display(img)

# Move models to device
unet.to(DEVICE)
vqvae.to(DEVICE)

# Set models to training mode
unet.train()
vqvae.train()

# Define optimizer (only parameters that require gradients)
optimizer = optim.Adam(
    list(unet.parameters()) + list(vqvae.parameters()),
    lr=LEARNING_RATE
)

# Define a loss function, e.g., Mean Squared Error
criterion = nn.MSELoss()

# Training Loop
for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    epoch_loss = 0.0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)
    
    for batch_idx, batch in enumerate(progress_bar):
        # Move batch to device
        images = batch.to(DEVICE)
        
        # Forward pass through VQ-VAE to get latent representations
        with torch.no_grad():
            vqvae_output = vqvae.encode(images)
            # Corrected line: Use 'latent_sample' instead of 'latent_dist.sample()'
            latents = vqvae_output.latents
            latents = latents * vqvae.config.scaling_factor
        
        # Add noise according to the scheduler
        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, scheduler.num_train_timesteps, (latents.shape[0],), device=DEVICE).long()
        noisy_latents = scheduler.add_noise(latents, noise, timesteps)
        
        # Forward pass through U-Net
        noise_pred = unet(noisy_latents, timesteps).sample
        
        # Compute loss
        loss = criterion(noise_pred, noise)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Mask gradients of pruned weights
        with torch.no_grad():
            for module in unet.modules():
                if isinstance(module, nn.Conv2d) and module.weight.grad is not None:
                    zero_mask = module.weight == 0
                    if zero_mask.any():
                        module.weight.grad[zero_mask] = 0
            for module in vqvae.modules():
                if isinstance(module, nn.Conv2d) and module.weight.grad is not None:
                    zero_mask = module.weight == 0
                    if zero_mask.any():
                        module.weight.grad[zero_mask] = 0
        
        # Optimizer step
        optimizer.step()
        
        # Re-mask the weights to ensure pruned weights remain zero
        with torch.no_grad():
            for module in unet.modules():
                if isinstance(module, nn.Conv2d):
                    module.weight[module.weight == 0] = 0
            for module in vqvae.modules():
                if isinstance(module, nn.Conv2d):
                    module.weight[module.weight == 0] = 0
        
        # Accumulate loss
        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    
    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f"Average Loss: {avg_epoch_loss:.4f}")
    
    # Save the model at each epoch
    torch.save(unet.state_dict(), FINE_TUNED_UNET_PATH_TEMPLATE.format(epoch+1))
    torch.save(vqvae.state_dict(), FINE_TUNED_VQVAE_PATH_TEMPLATE.format(epoch+1))
    print(f"Saved fine-tuned models for epoch {epoch+1}")
    
    # Uncomment the following line to generate sample images after training
    generate_sample_images(pipeline, num_images=1, prompt="A high quality portrait")


print("\nTraining complete!")



2024-12-06 10:25:58.198445: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-06 10:25:58.424745: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 10:25:58.424810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 10:25:58.446471: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 10:25:58.496414: I tensorflow/core/platform/cpu_feature_guar

Loading pre-trained models and scheduler...


An error occurred while trying to fetch CompVis/ldm-celebahq-256: CompVis/ldm-celebahq-256 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/home/cqilab/anaconda3/envs/sgrs/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
An error occurred while trying to fetch CompVis/ldm-celebahq-256: CompVis/ldm-celebahq-256 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/home/cqilab/anaconda3/envs/sgrs/lib/python3.11/site-packages/diffusers/config

Sparsity of U-Net: 41.79% (114457504/273915936)
Sparsity of VQ-VAE: 48.07% (26575752/55287954)
Initializing the LDMPipeline with pruned models...

Epoch 1/2


Training:   0%|          | 0/938 [00:00<?, ?it/s]/home/cqilab/anaconda3/envs/sgrs/lib/python3.11/site-packages/diffusers/configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDIMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDIMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
Training:  17%|█▋        | 162/938 [04:16<20:31,  1.59s/it, loss=0.0192]

In [ ]:
import os
from PIL import Image

result_folder = "result_70"
def generate_sample_images(pipeline, num_images=10, num_inference_steps_list=[20, 100], num_samples=50000):
    # Ensure the pipeline is on the correct device

    # Iterate over each number of inference steps (10, 20, 100)
    for num_inference_steps in num_inference_steps_list:
        # Create a subfolder for each number of inference steps
        steps_folder = os.path.join(result_folder, f"{num_inference_steps}_steps")
        if not os.path.exists(steps_folder):
            os.makedirs(steps_folder)

        # Generate images in batches, aiming for the total number of images
        num_batches = num_samples // num_images
        for batch_idx in range(num_batches):
            with torch.no_grad():
                # Generate images using the pipeline
                generated_images = pipeline(batch_size=num_images, num_inference_steps=num_inference_steps).images

            # Save each generated image to the corresponding subfolder
            for i, img in enumerate(generated_images):
                # Create a filename for the image
                img_path = os.path.join(steps_folder, f"generated_image_{batch_idx * num_images + i + 1}.png")
                
                # Save the image
                img.save(img_path)
                print(f"Image {batch_idx * num_images + i + 1} saved at: {img_path}")

# Example usage:
generate_sample_images(pipeline, num_images=10, num_inference_steps_list=[20, 100, 50], num_samples=50000)

